In [ ]:
from datetime import date
import uuid
import jaydebeapi, os
import random


def getToday():
    return date.today().isoformat()

def generateSQL(tablename, columns, values):
    
    sql= ' insert into ' + tablename + ' (' + columns +  ' ) values (' + values + ');'
    
    return sql

In [ ]:
class Customer:
    
    columns = ['customerNumber',
     'customerName',           
    'phoneNumber',
    'postalCode',
    'locale',
    'dateCreated',
    'email']
    
    salesOrders = []

    
    def __init__(self):
        
        self.customerNumber = uuid.uuid1()
        self.customerName = randomString(20)
        self.phoneNumber = random.randint(1000000,9000000)
        self.postalCode = randomString(4)
        self.locale = 'en-US.utf-8'
        self.dateCreated = getToday()
        self.email = randomString(4) + '@' + randomString(4) + ".com"
        
        for i in range(random.randint(1,5)):
            self.salesOrders.append(SalesOrder(self.customerNumber))
        
    def getColumns(self):
        return self.columns
    
    def getValues(self):
               
        return [self.customerNumber,
            self.customerName,
            self.phoneNumber,
            self.postalCode,
            self.locale,
            self.dateCreated,
            self.email]

    def getSalesOrders(self):
        return self.salesOrders
     
    
'''    
create table customers (
customerNumber uuid,
customerName varchar(50),
phoneNumber varchar(14),
postalCode varchar(4),
locale varchar(11),
dateCreated timestamp,
email varchar(20));
'''        
        

#class InventoryMovement:
    

#class Product:
    


class SalesOrder:

    columns =  ['customerNumber',
        'orderNumber' ,
        'comments' ,
        'orderDate' ,
        'orderType' ,
        'shipDate'  ,
        'discount'  ,
        'quantity'  ,
        'productNumber']

    def __init__(self, customerNumber):

        self.customerNumber = customerNumber
        self.orderNumber = uuid.uuid1()
        self.comments = randomString(20)
        self.orderDate =  getToday()
        self.orderType = "sale"
        self.shipDate  =  getToday() 
        self.discount  = 0
        self.quantity  = random.randint(1,100)
        self.productNumber = uuid.uuid1()  
        
    def getColumns(self):
        return self.columns    
        
        
    def getValues(self):
        return  [ self.customerNumber,
        self.orderNumber ,
        self.comments ,
        self.orderDate ,
        self.orderType ,
        self.shipDate  ,
        self.discount  ,
        self.quantity  ,
        self.productNumber ]

'''
create table salesOrder (
    customerNumber uuid,
    orderNumber uuid,
    comments varchar(200),
    orderDate timestamp,
    orderType varchar(20),
    shipDate timestamp,
    discount float,
    quantity integer,
    productNumber varchar(50));
'''


In [ ]:


def connect():
    dsn_database = "inventory"
    dsn_hostname = "paris2x"
    dsn_port = "5432"
    dsn_uid = "walker"
    dsn_pwd = "livres"
    jdbc_driver_name = "org.postgresql.Driver"
    jdbc_driver_loc = os.path.join('/home/ubuntu/Documents/postgresql-42.2.14.jar')

    sql_str = "select version()"

    connection_string='jdbc:postgresql://'+ dsn_hostname+':'+ dsn_port +'/'+ dsn_database

    url = '{0}:user={1};password={2}'.format(connection_string, dsn_uid, dsn_pwd)
    print("Connection String: " + connection_string)

    conn = jaydebeapi.connect(jdbc_driver_name, connection_string, {'user': dsn_uid, 'password': dsn_pwd},
    jars=jdbc_driver_loc)

    return conn


In [ ]:


def randomString(length):
    
    char=""
    for i in range(length):
        char = char + chr(random.randint(97,123))
    return char

In [ ]:
def commaCheck(field):
    
     if isinstance(field, uuid.UUID):
        return   str(field)  
    
     if isinstance(field, int):
        return  str(field)  
    
     if isinstance(field, str):
        return field 

In [ ]:
def generateCustomers(conn):
    for i in range(2000):
        
        customer = Customer()
       
        columns = ','.join([col for col in customer.getColumns()])
        
        values = ','.join(["'" + commaCheck(value) + "'" for value in customer.getValues()])

        sql=generateSQL("customers",columns,values)
       

        curs = conn.cursor()
        curs.execute(sql)
        
        generateOrders(customer.getSalesOrders(),conn)
       
        

In [ ]:
def generateOrders(orders,conn):
    for order in orders:
         
        columns = ','.join([col for col in order.getColumns()])
        
        values = ','.join(["'" + commaCheck(value) + "'" for value in order.getValues()])

        sql=generateSQL("salesOrder",columns,values)
   
        
        curs = conn.cursor()
        curs.execute(sql) 

         

In [ ]:
def main():
    conn=connect()
    generateCustomers(conn)
   

if __name__ == "__main__":
    main()